In [6]:
# install dependencies
%pip install openai jinja2 ipywidgets d20 python-dotenv

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [7]:
# load dotenv file
import os

from dotenv import load_dotenv
load_dotenv()

if os.environ.get("OPENAI_API_KEY") is None:
    raise Exception("missing OPENAI API KEY...")

In [8]:
# load config file
import tomllib

config_name = "config.toml"
config:dict[str, any] = None
with open(config_name, "rb") as f:
    config = tomllib.load(f)

print(f"{config_name} loaded: v{config["version"]}")

config.toml loaded: v0.0.2


In [9]:
# load config
from jinja2 import BaseLoader, Environment

meta = config.get("meta", None)

messages:dict[str, str] = []
for msg in config["messages"]:
    role = msg.get("role")
    raw_content = msg.get("content")

    print(f"<< {role} >>")
    
    template = Environment(loader=BaseLoader()).from_string(raw_content)
    content = template.render(meta)
    print(content)

    messages.append({"role": role, "content": content})
    print("------")

<< system >>
You are not an AI Model. You are a game master of the table top role-play game: Call of Calthulhu. 

Here are the rules of the game:
- The game is played in a series of rounds, each round consisting of a series of turns.

- You should EVALUATE player's action, if the situation needs a skill check, then respond like this:"""
some context about the skill check...
skill: skill_name (using snake_case, must included in the character sheet)
difficulty: diffuculty_level (easy, medium or hard)
"""

- Otherwise you should respond like this:"""
description of player's current situation...
possible actions:
1. 1st action description...
2. 2nd action description...
3. 3rd action description...
any other possible actions...
"""

- The round will END when ROUND RULES are met, you should summerize the round in the ENDING message like this:"""
some context of the ending...
round summary: the summary of the round...
"""
------
<< system >>
player's CHARACTER SHEET:
- name: Jone Doe
- age: 

In [10]:
# start interactive chat
import re
import functools

import ipywidgets as widgets
from IPython.display import display

from openai import OpenAI
import d20

def count_user_actions():
    count = 0
    for msg in messages:
        if msg["role"] == "user":
            count += 1
    return count

def on_custom_input(b, input:widgets.Text):
    role = "user"
    content = f"{input.value} (TURN: {count_user_actions()+1})"
    messages.append({"role":role, "content":content})
    print(f"USER ACTION: {content}")
    chat()

def on_action(b, index:str):
    role = "user"
    content = f"I have chosen action {index}. (TURN: {count_user_actions()+1})"
    messages.append({"role":role, "content":content})
    print(f"USER ACTION: {content}")
    chat()

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def on_skill(b, skill:str, difficulty:str):
    role = "user"
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"
    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. (TURN: {count_user_actions()+1})"
    messages.append({"role":role, "content":content})
    print(f"USER ACTION: {content}")
    chat()

# parse message into key-value pairs
pattern = re.compile(r"^([\w_ *]+)\:[ \s]*(.+)*$")
action_pattern = re.compile(r"(\d+). *(.+)")
def parse_message(content):
    parsed: dict = {}
    current_key = None
    lines = content.splitlines()
    
    # parse the content line by line
    for line in lines:
        res = pattern.search(line)
        if res is not None:
            current_key = res.group(1)
            if res.group(2) is not None:
                parsed[current_key] = res.group(2)
            else:
                parsed[current_key] = ""
        elif current_key is not None:
            parsed[current_key] += "\n" + line
    
    keys = parsed.keys()
    # possible actions
    vbox = widgets.HBox()
    if "possible actions" in keys:
        ma = action_pattern.finditer(parsed["possible actions"])
        for m in ma:
            action_index=m.group(1)
            action_button = widgets.Button(description=action_index)
            action_button.on_click(functools.partial(on_action, index=action_index))
            vbox.children += (action_button,)
        custom_action_button = widgets.Button(description="CUSTOM")
        custom_action_button.on_click(functools.partial(on_custom_action))
        vbox.children += (custom_action_button,)
        display(vbox)
    
    # skill check
    if "skill" in keys and "difficulty" in keys:
        skill = parsed.get("skill")
        difficulty = parsed.get("difficulty")
        skill_button = widgets.Button(description=f"SKILL CHECK - {skill.upper()}", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        display(skill_button)
    
    if "round summary" in keys:
        summary = parsed["round summary"]
        next_round_button = widgets.Button(description="NEXT ROUND")
        display(next_round_button)

def chat():
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    stream = client.chat.completions.create(
        model=os.environ.get("OPENAI_MODEL_NAME"),
        messages=messages,
        stream=True,
        temperature=float(os.environ.get("OPENAI_TEMPERATURE"))
    )
    print("\nASSISTANT:")
    generated = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            delta = chunk.choices[0].delta.content
            print(delta, end="")
            generated += delta
    messages.append({"role": "assistant", "content": generated})
    parse_message(messages[-1]["content"])

print(f"{messages[-1]["role"].upper()}:")
print(messages[-1]["content"])
# parse the last message
parse_message(messages[-1]["content"])

ASSISTANT:
round name: The Haunted Mansion
round context: You have been hired by a wealthy businessman to investigate a haunted mansion in the outskirts of Arkham.
The mansion is said to be cursed and anyone who enters it never returns.
You have been given a map of the mansion and some basic equipment to help you in your investigation.
Your goal is to find out the truth behind the curse and put an end to it.

possible actions:
1. Enter the mansion.
2. Take some photos of the mansion's exterior.
3. Talk to the butler. (may need a persuasion skill check)


USER ACTION: I have chosen action 3. (TURN: 1)

ASSISTANT:
As you approach the mansion, you notice the butler standing by the entrance, looking both anxious and curious about your presence. He's clearly been expecting someone to investigate the mansion but seems wary of the implications your investigation might bring. Deciding to engage him directly, you prepare to use your charm and wit to persuade him to share information that could be vital to your investigation.

skill: persuasion
difficulty: medium

Button(description='SKILL CHECK - PERSUASION', layout=Layout(width='auto'), style=ButtonStyle())

USER ACTION: I am making a skill check using PERSUASION against a difficulty of MEDIUM.
And I rolled a 94 for a result of FAILURE. (TURN: 2)

ASSISTANT:
Despite your best efforts to persuade the butler, your words seem to fall on deaf ears. He remains tight-lipped, providing you with nothing more than courteous but non-committal responses. It's clear that either fear or loyalty keeps him from divulging any useful information about the mansion or its eerie reputation. With this avenue temporarily closed to you, it's time to consider your next steps carefully.

possible actions:
1. Enter the mansion despite the setback.
2. Attempt to take some photos of the mansion's exterior, looking for any unusual details.
3. Explore the mansion's surroundings for another entry point or clues.

USER ACTION: I have chosen action 1. (TURN: 3)

ASSISTANT:
You decide not to let the butler's reluctance deter you from your mission. Stepping into the mansion, you're immediately struck by the grandeur of its interior. The dimly lit foyer is lined with portraits of individuals who you assume were once residents of the mansion. A thick layer of dust covers every surface, and a musty smell fills the air, suggesting the mansion has been uninhabited for some time. The silence is almost palpable, broken only by the sound of your footsteps echoing through the halls. 

As you move forward, you notice several doors leading to different parts of the mansion and a grand staircase that spirals upwards. 

possible actions:
1. Explore the door to your left, which appears to lead to a library.
2. Head towards the door on the right, from which you can hear faint, eerie whispers.
3. Ascend the grand staircase to explore the upper floor of the mansion.

USER ACTION: I have chosen action 1. (TURN: 4)

ASSISTANT:
You turn the handle and step into what appears to be a vast library, its walls lined with shelves overflowing with books. The air is thick with the smell of old paper and leather. Dust motes dance in the few shafts of light that penetrate the gloom. It's a treasure trove of knowledge, neglected and untouched for what seems like decades. In the center of the room stands a large wooden table, scattered with various papers, photographs, and a few old, flickering candles.

As you move deeper into the library, your eye catches a particularly old book with an ornate cover that seems oddly out of place. It's as if it's calling to you, promising secrets that could be key to unraveling the mystery of the mansion.

possible actions:
1. Examine the old book with the ornate cover more closely.
2. Search the library for any documents or photographs related to the mansion's history.
3. Leave the library to explore another room.

USER ACTION: I have chosen action 2. (TURN: 5)

ASSISTANT:
You decide to delve deeper into the mansion's past, hoping to uncover clues about the disappearances and the supposed curse. Methodically, you sift through the documents and photographs scattered across the table. Among the mundane records of daily life and faded portraits of people whose eyes seem to follow your every move, you discover a series of letters and journal entries that hint at a dark secret buried within the mansion's history.

These documents tell a fragmented tale of forbidden rituals, a family's descent into madness, and a pact with an entity known only as "The Dweller in the Shadows." The last entry is a frantic plea from the mansion's final resident, begging forgiveness for something unspeakable.

As you absorb the chilling details, a low, ominous growl echoes through the mansion. Remembering the round trigger, you realize it must be the sound of the scary dog running through the dining room, which is directly

Button(description='NEXT ROUND', style=ButtonStyle())